**Test physical form of tuned equation on ERA5 data**

*Delayed*

In [3]:
import os
import sys
import json
import numpy as np
import xarray as xr
import sympy as sp
import matplotlib.pyplot as plt

subset_exp = 2
# subset_exp = 2
number_horizontal_locations = 10**subset_exp
tl_bool = True

sys.path.insert(0, os.environ['HOME'] + '/workspace_icon-ml/cloud_cover_parameterization/')
import my_classes
from my_classes import load_data

sys.path.insert(0, os.environ['HOME'] + '/workspace_icon-ml/symbolic_regression/')
from functions import add_derivatives
from functions import append_dict_to_json

SEED = 10

num_cells = 1368

**Load data**

In [6]:
order_of_vars = ['q', 'clwc', 'ciwc', 't', 'pa', 'zg', 'cc']
data_dict = load_data(source='era5', days='one', order_of_vars=order_of_vars)

TIMESTEPS, VLAYERS, HFIELDS = data_dict['q'].shape

# Removing four upper-most levels
for key in data_dict.keys():
    data_dict[key] = data_dict[key][:, 4:].copy()

# Data output
data_output = data_dict['cc']
del data_dict['cc']

for key in data_dict.keys():
    print(data_dict[key].shape)
    assert data_dict[key].shape == data_dict[key].shape

one
Assertion warning. Max cc not 100. Instead:
100.0
(24, 27, 66655)
(24, 27, 66655)
(24, 27, 66655)
(24, 27, 66655)
(24, 27, 66655)
(24, 27, 66655)


In [9]:
npy_file_names

['int_var_1979-01-01_rh_z_R02B05.npy',
 'int_var_1979-04-01_rh_z_R02B05.npy',
 'int_var_1979-07-01_rh_z_R02B05.npy',
 'int_var_1979-10-01_rh_z_R02B05.npy',
 'int_var_1980-01-01_rh_z_R02B05.npy',
 'int_var_1980-04-01_rh_z_R02B05.npy',
 'int_var_1980-07-01_rh_z_R02B05.npy',
 'int_var_1980-10-01_rh_z_R02B05.npy',
 'int_var_1981-01-01_rh_z_R02B05.npy',
 'int_var_1981-04-01_rh_z_R02B05.npy',
 'int_var_1981-07-01_rh_z_R02B05.npy',
 'int_var_1981-10-01_rh_z_R02B05.npy',
 'int_var_1982-01-01_rh_z_R02B05.npy',
 'int_var_1982-04-01_rh_z_R02B05.npy',
 'int_var_1982-07-01_rh_z_R02B05.npy',
 'int_var_1982-10-01_rh_z_R02B05.npy',
 'int_var_1983-01-01_rh_z_R02B05.npy',
 'int_var_1983-04-01_rh_z_R02B05.npy',
 'int_var_1983-07-01_rh_z_R02B05.npy',
 'int_var_1983-10-01_rh_z_R02B05.npy',
 'int_var_1984-01-01_rh_z_R02B05.npy',
 'int_var_1984-04-01_rh_z_R02B05.npy',
 'int_var_1984-07-01_rh_z_R02B05.npy',
 'int_var_1984-10-01_rh_z_R02B05.npy',
 'int_var_1985-01-01_rh_z_R02B05.npy',
 'int_var_1985-04-01_rh_z

In [8]:
import time
t0 = time.time()

# Add rh
T0 = 273.15
r = 0.00263*data_dict['pa']*data_dict['q']*np.exp((17.67*(data_dict['t']-T0))/(data_dict['t']-29.65))**(-1)
data_dict['rh'] = r

# Add rh_z
folder = 'rh_z'

# Initialize all_npy_files with empty tensor
all_npy_files = np.zeros((0, VLAYERS-4, HFIELDS))

# Load all filenames in the folder containing the derivatives. The filenames are sorted chronologically.
npy_file_names = sorted(os.listdir(os.path.join(os.environ['HOME'] + '/bd1179_work/ERA5/hvcg_data', folder)))        

for file in npy_file_names:
    # Load three-hourly data and convert directly to float32
    npy_file = np.load(os.environ['HOME'] + '/bd1179_work/ERA5/hvcg_data/%s/%s'%(folder,file), mmap_mode='r')
    npy_file = np.float32(npy_file[0::3].copy())
    all_npy_files = np.concatenate((all_npy_files, npy_file), axis=0)
data_dict[folder] = all_npy_files 

KeyboardInterrupt: 

**Reshaping and keeping only the relevant features**

In [ ]:
# Keeping only the relevant features
features = ['rh', 't', 'clwc', 'ciwc', 'rh_z']
for key in features:
    data_dict[key] = np.reshape(data_dict[key], -1)
    
data_output = np.reshape(data_output, -1)

del data_dict['q']
del data_dict['pa']
del data_dict['zg']

no_features = len(data_dict.keys())

**Cast dict into ndarray**

In [ ]:
# data_array = np.zeros((data_dict['q'].size, len(data_dict.keys())), dtype=np.float32)

k = 0
data_array_not_T = []
for key in features:
    print(key)
    data_array_not_T.append(np.reshape(data_dict[key], -1))
    del data_dict[key]
    k += 1

# Convert into np array and transpose
data_array = np.transpose(np.array(data_array_not_T, dtype=np.float32))

In [ ]:
# Update loc
loc = {}
for i in range(len(features)):
    loc[features[i]] = i

**Pick the subset**

In [ ]:
seed = np.random.seed(SEED)
subset = np.random.randint(0, HFIELDS, number_horizontal_locations)
# Convert to regular int to make check_sum JSON serializable
check_sum = int(np.sum(subset))

# Collecting all grid cell indices for the horizontal fields given by subset
Z = np.zeros((TIMESTEPS, 27, HFIELDS), dtype=int)
for k in range(HFIELDS):
    Z[:,:,k] = k
Z_res = np.reshape(Z, -1)
subset_inds = np.concatenate([np.where(Z_res == s)[0] for s in subset])

In [ ]:
train_input = data_array[subset_inds[:num_cells]] #num_hours*27
train_output = data_output[subset_inds[:num_cells]] #num_hours*27

**Already remove the regime with clw + cli = 0**

In [ ]:
reg_0 = np.where(data_array[:, loc['clwc']] + data_array[:, loc['ciwc']] <= 1e-20)[0]
reg_not_0 = np.where(data_array[:, loc['clwc']] + data_array[:, loc['ciwc']] > 1e-20)[0]

# Relevant values to compute final MSE/R2-scores
mse_reg_0 = np.mean(data_output[reg_0]**2)
len_reg_0 = len(reg_0)
len_reg_not_0 = len(reg_not_0)
len_data_output = len(data_output)
var_data_output = np.var(data_output)

data_array = data_array[reg_not_0]
data_output = data_output[reg_not_0]

In [ ]:
print(mse_reg_0)
print(data_array.shape)
print(data_output.shape)

# Should be 338023
len_reg_0

Optimize coefficients

In [ ]:
# See ~/symbolic_regression/finding_symmetries/pysr_results_dyamond_on_regimes/optimize_coefs_EQ4.ipynb
def func(X, a,b,c,d,e,f,g,h,i,j):
    # features = ['rh', 't', 'clwc', 'ciwc', 'rh_z']
    
    rh = X[:, 0]
    ta = X[:, 1]
    clwc = X[:, 2]
    ciwc = X[:, 3]
    dzrh = X[:, 4]
    
    a1,a2,a3,a4,a5,a6,a7,a8,a9,eps = [0.31229407460832626, 1.1736956030268146, -0.013983869134317005, 2.6452281700144, 0.000430595940803418, 291.62153962516146, \
                                      32.3312283084943, 6.4948778178700745e-06, 1.8527761701740456e-06, 1.11263027094115]
    
    RH0 = 0.6025
    T0 = 257.06
    
    I1 = a1 + a2*(rh - RH0) + a3(ta - T0) + a4/2*(rh - RH0)**2 + a5/2*(ta - T0)**2*(rh - RH0)
    I2 = a6**3*(dzrh + 0.1/a7)*dzrh**2
    I3 = -1/(clwc/a8 + ciwc/a9 + eps)
    
    return I1 + I2 + I3

In [ ]:
import scipy as sci
from scipy.optimize import minimize

In [ ]:
def objective(X,Y):
    '''
        The objective function.
    '''
    train_preds = np.minimum(np.maximum(100*func(X, a,b,c,d,e,f,g,h,i,j), 0), 100)
    train_mse = np.mean((train_preds - Y)**2, dtype=np.float64)

    return train_mse

mse_reg_1 = objective(data_scaled, data_output)

results = {}

mse_new_total = (mse_reg_0*len_reg_0 + mse_reg_1*len_reg_not_0)/len_data_output
r2_new_total = 1 - mse_new_total/var_data_output

print(mse_new_total, r2_new_total)

parent_key = 'pysr_EQ4_no_tl'
results[parent_key] = {}
results[parent_key]['MSE'] = mse_new_total
results[parent_key]['R2'] = r2_new_total
results[parent_key]['Coefficients'] = list(res.x)
# Should be the same for all runs
results[parent_key]['Check_sum'] = check_sum

In [ ]:
results